# Data validation with Cats

## The Problem

We got a requirement to validate user data coming from a Front End Registration form

In [10]:
interp.configureCompiler(_.settings.YpartialUnification.value = true)

case class UserDTO(email: String, password: String)

private val emailRegex =
    """^[a-zA-Z0-9\.!#$%&'*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?)*$""".r

private val passwordRegex = """[a-zA-Z0-9]+"""

def isValidEmail(email: String): Boolean = email match {
    case null                                          => false
    case e if e.trim.isEmpty                           => false
    case e if emailRegex.findFirstMatchIn(e).isDefined => true
    case _                                             => false
}

def isValidPassword(password: String): Boolean = password match {
    case null                                          => false
    case p if p.trim.isEmpty                           => false
    case p if p.matches(passwordRegex) && p.length > 5 => true
    case _                                             => false
}

defined class UserDTO
defined function isValidEmail
defined function isValidPassword

## Version 0 Naive Implementation

Naive implementation that just throws, when validation fails

In [11]:
class UserValidationException extends Exception("User validation exception")

def validateUserVersion0(user: UserDTO): UserDTO =
  if (isValidEmail(user.email) && isValidPassword(user.password)) {
    user
  } else {
    throw new UserValidationException
  }

defined class UserValidationException
defined function validateUserVersion0

## Version 1 Smart Constructors

In [12]:
case class Email(value: String)
object Email {
  def apply(email: String): Option[Email] = 
    Some(email).filter(isValidEmail).map(new Email(_))
}

case class Password(value: String)
object Password {
  def apply(password: String): Option[Password] = 
    Some(password).filter(isValidPassword).map(new Password(_))
}

case class User(email: Email, password: Password)

object User {
  def apply(email: Email, password: Password): User = new User(email, password)

  def fromUserDTO(user: UserDTO): Option[User] = for {
    email <- Email(user.email)
    password <- Password(user.password)
  } yield new User(email, password)
}

def validateUserVersion1(user: UserDTO): Option[User] = User.fromUserDTO(user)

defined class Email
defined object Email
defined class Password
defined object Password
defined class User
defined object User
defined function validateUserVersion1

## Version 2 Transforming to Either

In [13]:
val userError = "User validation error"
def validateUserVersion2(user: UserDTO): Either[String, User] = 
    User.fromUserDTO(user).toRight(userError)

userError: String = "User validation error"
defined function validateUserVersion2

## Version 3 Combining Errors

In [14]:
val emailError = "invalid email"
val passwordError = "invalid password"

def validateUserVersion3(user: UserDTO): Either[String, User] = (
    Email(user.email).toRight(emailError),
    Password(user.password).toRight(passwordError)
  ) match {
    case (Right(email), Right(password)) => Right(User(email, password))
    case (Left(error), Right(_))         => Left(error)
    case (Right(_), Left(error))         => Left(error)
    case (Left(e1), Left(e2))            => Left(e1 ++ e2)
}

emailError: String = "invalid email"
passwordError: String = "invalid password"
defined function validateUserVersion3

## Version 4 Simplify Syntax

In [15]:
def validateUserVersion4(user: UserDTO): Either[String, User] = for {
  email <- Email(user.email).toRight(emailError)
  password <- Password(user.password).toRight(passwordError)
} yield User(email, password)

defined function validateUserVersion4

## Version 5 New Syntax

In [18]:
import $ivy.`org.typelevel::cats-core:1.6.0`
import cats.data.Validated
import cats.data.Validated.{Invalid, Valid}
import cats.implicits._

def validateUserVersion5(user: UserDTO): Validated[String, User] = (
  Email(user.email).toValid(emailError),
  Password(user.password).toValid(passwordError)
).mapN(User(_, _))

import $ivy.$                               

import cats.data.Validated

import cats.data.Validated.{Invalid, Valid}

import cats.implicits._


defined function validateUserVersion5

## Version 6 Modeling Dependent Errors

In [16]:
import cats.implicits._
import cats.data.NonEmptyList
import cats.data.ValidatedNel
import cats.data.Validated
import cats.data.Validated.{Invalid, Valid}

sealed trait UserError
final case object PasswordValidationError extends UserError

sealed trait EmailError extends UserError
final case object InvalidEmailError extends EmailError
final case object BlackListedUserError extends EmailError

val blackListedUsers = Seq("bart@simsom.com")

def validatedEvilness(email: Email): ValidatedNel[UserError, Email] =
 Validated.condNel(!blackListedUsers.contains(email.value), 
                   email, 
                   BlackListedUserError)


def validateUserVersion6(user: UserDTO): ValidatedNel[UserError, User] = (
  Email(user.email).toValidNel(InvalidEmailError)
                   .andThen(validatedEvilness),
  Password(user.password).toValidNel(PasswordValidationError)
).mapN(User(_, _))

import cats.implicits._

import cats.data.NonEmptyList

import cats.data.ValidatedNel

import cats.data.Validated

import cats.data.Validated.{Invalid, Valid}


defined trait UserError
defined object PasswordValidationError
defined trait EmailError
defined object InvalidEmailError
defined object BlackListedUserError
blackListedUsers: Seq[String] = List("bart@simsom.com")
defined function validatedEvilness
defined function validateUserVersion6

## Version 7 Generalizing

In [17]:
def validateUserVersion1(user: UserDTO) = validateUserVersion6(user).toOption

def validateUserVersion3(user: UserDTO) = validateUserVersion6(user).toEither

defined function validateUserVersion1
defined function validateUserVersion3